## Fetch Vaccine List

In [2]:
import psycopg2
from psycopg2.extras import execute_values
from faker import Faker
import random

conn = psycopg2.connect(
    host="aws-1-ap-south-1.pooler.supabase.com",
    database="postgres",
    user="postgres.rruavcjmtgpxyznzwkhw",
    password="khaibaolocnguyen",
    port=5432
)
cur = conn.cursor()

fake = Faker("vi_VN")

In [3]:
cur.execute("""
  select vaccine_id from vaccine
""")

vaccine_ids = cur.fetchall()
vaccine_ids = [e[0] for e in vaccine_ids]
print(len(vaccine_ids))

10029


## Tạo package

In [4]:
package_names = [
    "Basic Vaccination Package",
    "Advanced Vaccination Package",
    "Comprehensive Vaccination Package",
    "Regular Health Check Package",
    "Pet Disease Prevention Package",
    "Puppy & Kitten Vaccination Package",
    "Annual Vaccination Package",
    "Immune Protection Package",
    "Digestive System Vaccination Package",
    "Oral Care Package",
    "Dermatology Prevention Package",
    "General Health Checkup Package",
    "Viral Vaccination Package",
    "Immune Boost Package",
    "Parasite Prevention Package",
    "Premium Pet Care Package",
    "Vaccination Reminder & Support Package",
    "Infectious Disease Prevention Package",
    "Puppy Care Package",
    "Kitten Care Package",
    "Periodic Immunity Check Package",
    "First Month Vaccination Package",
    "Spring Vaccination Package",
    "Summer Vaccination Package",
    "Autumn Vaccination Package",
    "Winter Vaccination Package",
    "Comprehensive Health Protection Package",
    "Pet Flu Vaccination Package",
    "Antibody Boost Package",
    "Advanced Pet Vaccination Package",
    "Senior Pet Wellness Package", "Feline Leukemia Prevention Package", "Canine Parvovirus Defense Package",
    "Rabies Protection Essential", "Travel Readiness Vaccination Package", "Shelter & Rescue Support Package",
    "Active Dog Performance Package", "Indoor Cat Security Package", "Respiratory Health Protection",
    "Multi-Pet Household Discount Package", "Post-Surgery Recovery Package", "Heartworm & Lyme Prevention Bundle",
    "Breeder's First Choice Package", "Allergy Management & Prevention", "New Adoption Welcome Package",
    "Holistic Immunity Wellness", "Urban Pet Hazard Package", "Liver & Kidney Health Support",
    "Seasonal Allergy & Flu Guard", "Elite Lifetime Protection Plan"
]

package_descriptions = [
    "Includes basic vaccination shots to prevent common diseases in pets.",
    "Advanced vaccination package to enhance immunity and provide long-term protection.",
    "Comprehensive health care with all necessary vaccination shots.",
    "Suitable for pets requiring regular checkups and vaccinations as recommended by veterinarians.",
    "Effectively prevents common infectious diseases in dogs and cats.",
    "Specifically designed for puppies and kittens during early development stages.",
    "Annual vaccination package to maintain stable pet health.",
    "Boosts immunity and reduces the risk of serious illnesses.",
    "Prevents diseases related to the digestive system and intestines.",
    "Maintains oral health, preventing gum inflammation and tooth decay.",
    "Protects skin and coat from common dermatological issues.",
    "General health checkup and advice for proper pet care.",
    "Vaccination against common viruses to keep pets healthy.",
    "Strengthens the immune system for pets of all ages.",
    "Prevents internal and external parasites effectively.",
    "Premium care service, monitoring pet health comprehensively.",
    "Supports scheduled vaccinations with reminders and professional advice.",
    "Helps prevent dangerous infectious diseases.",
    "Specialized care for puppies to ensure healthy development.",
    "Specialized care for kittens to protect overall health.",
    "Periodic immunity check to evaluate vaccination effectiveness.",
    "First-month vaccination package, protecting initial immunity.",
    "Spring vaccination package, boosting pets' resistance for the season.",
    "Summer vaccination package, reducing risk of heat-related illnesses.",
    "Autumn vaccination package, preparing immunity for winter.",
    "Winter vaccination package, protecting pets from cold and flu.",
    "Comprehensive health protection package to prevent multiple diseases.",
    "Pet flu vaccination package, minimizing outbreak risks.",
    "Antibody boost package to maintain long-term pet health.",
    "Advanced vaccination package offering optimal protection for pets of all ages.",
    "Specialized care and vaccinations tailored for aging pets to maintain quality of life.",
    "Targeted protection against FeLV for outdoor cats or multi-cat households.",
    "High-intensity immunity boost specifically against the deadly Parvovirus in dogs.",
    "Focuses on legal compliance and essential rabies prevention for all pets.",
    "Prepares pets for domestic or international travel with necessary certifications.",
    "Affordable vaccination bundle designed for high-density environments and rescues.",
    "Immunity and joint support for working dogs or pets with high physical activity.",
    "Essential vaccinations focused on risks common to strictly indoor feline environments.",
    "Prevents kennel cough and other airborne infections in social pet settings.",
    "Cost-effective vaccination solution for families with three or more pets.",
    "Immune support and health monitoring following major surgical procedures.",
    "Comprehensive defense against vector-borne diseases from mosquitoes and ticks.",
    "Bulk vaccination and health screening for professional breeders and litters.",
    "Focuses on skin immunity and reducing sensitivity to environmental allergens.",
    "Complete screening and initial shots for pets transitioning to a new home.",
    "Combines traditional vaccines with nutritional advice for balanced pet health.",
    "Protects against diseases common in city environments, like Leptospirosis.",
    "Vaccination adjustments and screenings for pets with metabolic sensitivities.",
    "Dynamic protection that adjusts to current local disease outbreaks and seasons.",
    "Our most premium package covering all annual boosters and emergency health consults."
]


In [5]:
packages = []
package_ids = []

for _ in range(1000):
  package_name = random.choice(package_names)
  duration = random.choice([30, 60, 90, 180, 365])
  description = random.choice(package_descriptions)
  discount_rate = random.choice([0, 5, 10, 15, 20, 25, 30])
  total_original_price = random.randint(500000, 5000000)

  package = {
    "package_name": package_name,
    "duration": duration,
    "description": description,
    "discount_rate": discount_rate,
    "total_original_price": total_original_price
  }

  packages.append(package)

print(len(packages))

1000


In [6]:
query = """
INSERT INTO vaccinationpackage (package_name, duration, description, discount_rate, total_original_price)
VALUES %s
RETURNING package_id
"""

values = [
    (p["package_name"], p["duration"], p["description"], p["discount_rate"], p["total_original_price"])
    for p in packages
]

execute_values(cur, query, values, page_size=len(values))

package_ids = [row[0] for row in cur.fetchall()]

conn.commit()

## Duyệt từng package, add Package Schedule

In [10]:
conn.rollback()

schedule_len_option = [1, 2, 3, 4, 5]

for pid in package_ids:
  schedule_len = random.choice(schedule_len_option)
  schedules = []
  for _ in range(schedule_len):
    schedule = {
      "schedule_week": random.randint(2, 16),
      "dosage": random.randint(5, 40),
      "vaccine_id": random.choice(vaccine_ids),
      "package_id": pid
    }
    schedules.append(schedule)

  query = """
  INSERT INTO vaccinationschedule (scheduled_week, dosage, vaccine_id, package_id)
  VALUES %s
  """

  values = [
      (s["schedule_week"], s["dosage"], s["vaccine_id"], s["package_id"])
      for s in schedules
  ]

  execute_values(cur, query, values, page_size=len(values))

  conn.commit()
